# Problem 7: Request Routing in a Web Server with a Trie

For this exercise we are going to implement an HTTP Router like you would find in a typical web server using the Trie data structure we learned previously.

There are many different implementations of HTTP Routers such as regular expressions or simple string matching, but the Trie is an excellent and very efficient data structure for this purpose.

The purpose of an HTTP Router is to take a URL path like "/", "/about", or "/blog/2019-01-15/my-awesome-blog-post" and figure out what content to return. In a dynamic web server, the content will often come from a block of code called a handler.

First we need to implement a slightly different Trie than the one we used for autocomplete. Instead of simple words the Trie will contain a part of the http path at each node, building from the root node /

In addition to a path though, we need to know which function will handle the http request. In a real router we would probably pass an instance of a class like Python's [SimpleHTTPRequestHandler](https://docs.python.org/3/library/http.server.html#http.server.SimpleHTTPRequestHandler) which would be responsible for handling requests to that path. For the sake of simplicity we will just use a string that we can print out to ensure we got the right handler

We could split the path into letters similar to how we did the autocomplete Trie, but this would result in a Trie with a very large number of nodes and lengthy traversals if we have a lot of pages on our site. A more sensible way to split things would be on the parts of the path that are separated by slashes ("/"). A Trie with a single path entry of: "/about/me" would look like:

(root, None) -> ("about", None) -> ("me", "About Me handler")

We can also simplify our RouteTrie a bit by excluding the suffixes method and the endOfWord property on RouteTrieNodes. We really just need to insert and find nodes, and if a RouteTrieNode is not a leaf node, it won't have a handler which is fine.

## RouteTrie and RouteTrieNode

In [1]:
# A RouteTrie will store our routes and their associated handlers
class RouteTrie:
    def __init__(self):
        # Initialize the trie with an root node and a handler, this is the root path or home page node
        self.root = RouteTrieNode()
        
    def _get_sub_paths(self, path):
        path = path.lower()
        # Checks for and delete trailing slash
        if len(path) > 0 and path[0] == '/':
            path = path[1:]
        if len(path) > 0 and path[-1] == '/':
            path = path[:-1]
            
        return path.split('/')

    def insert(self, full_path, handler):
        # Recursively adds nodes
        # Make sure you assign the handler to only the leaf (deepest) node of this path
        print("\n--- Inserting: ", str([full_path, handler]), " ---")
        sub_paths = self._get_sub_paths(full_path)
        node = self.root
        for path in sub_paths:
            node.insert(path)
            print(node)
            node = node.get_node(path)
        node.set_handler(handler)
        print(node)
        

    def find(self, path):
        # Starting at the root, navigate the Trie to find a match for this path
        # Return the handler for a match, or None for no match
        print("\n--- Finding: ", path, " ---")
        sub_paths = self._get_sub_paths(path)
        node = self.root
        for p in sub_paths:
            if p in node:
                node = node.get_node(p)
            else:
                return None
        return node.handler

# Represents a single node in RouteTrie
class RouteTrieNode:
    def __init__(self):
        # Initialize this node in the trie
        self.handler = None
        self.children = {}

    def insert(self, path):
        # Check for and delete trailing slash
        if path not in self:
            self.children[path] = RouteTrieNode()
        
    def get_node(self, path):
        return self.children[path]
    
    def set_handler(self, handler):
        self.handler = handler
            
    def __contains__(self, path):
        return path in self.children
    
    def __str__(self):
        s = "Handler: " + str(self.handler) + "\n"
        s += "Children: ["
        if len(self.children) == 0:
            s += "]"
            return s
        
        for i, child in enumerate(self.children):
            s += "'" + str(child) + "'"
            if i < len(self.children)-1:
                s += ", "
            else: 
                s += "]\n"
        return s

## Router

Next we need to implement the actual Router. The router will initialize itself with a RouteTrie for holding routes and associated handlers. It should also support adding a handler by path and looking up a handler by path. All of these operations will be delegated to the RouteTrie.

Hint: the RouteTrie stores handlers under path parts, so remember to split your path around the '/' character

Bonus Points: Add a not found handler to your Router which is returned whenever a path is not found in the Trie.

More Bonus Points: Handle trailing slashes! A request for '/about' or '/about/' are probably looking for the same page. Requests for '' or '/' are probably looking for the root handler. Handle these edge cases in your Router.

In [2]:
# The Router class will wrap the Trie and handle 
class Router:
    def __init__(self, root_handler, not_found_handler="404 - page not found"):
        # Create a new RouteTrie for holding our routes
        # You could also add a handler for 404 page not found responses as well!
        self.trie = RouteTrie()
        self.trie.insert('/', root_handler)
        self.not_found_handler = not_found_handler

    def add_handler(self, path, handler):
        # Add a handler for a path
        # You will need to split the path and pass the pass parts
        # as a list to the RouteTrie
        self.trie.insert(path, handler)
        

    def lookup(self, path):
        # lookup path (by parts) and return the associated handler
        # you can return None if it's not found or
        # return the "not found" handler if you added one
        # bonus points if a path works with and without a trailing slash
        # e.g. /about and /about/ both return the /about handler
        handler = self.trie.find(path)
        return handler if handler else self.not_found_handler

## Test Cases

### Test RouteTrie and RouteTrieNode

In [3]:
# Inserting a bunch of paths and handlers
routeTrie = RouteTrie()
routeTrie.insert('/', 'home_handler')
routeTrie.insert('/about/me/', 'about_me_handler')
routeTrie.insert('/sports', 'sports_handler')
routeTrie.insert('/career', 'career_handler')
routeTrie.insert('/sports/climbing', 'sports_climbing_handler')
routeTrie.insert('/sports/BJJ', 'sports_bjj_handler')
routeTrie.insert('/sports', 'NEW_sports_handler')
routeTrie.insert('/sports/judo', 'sports_judo_handler')


--- Inserting:  ['/', 'home_handler']  ---
Handler: None
Children: ['']

Handler: home_handler
Children: []

--- Inserting:  ['/about/me/', 'about_me_handler']  ---
Handler: None
Children: ['', 'about']

Handler: None
Children: ['me']

Handler: about_me_handler
Children: []

--- Inserting:  ['/sports', 'sports_handler']  ---
Handler: None
Children: ['', 'about', 'sports']

Handler: sports_handler
Children: []

--- Inserting:  ['/career', 'career_handler']  ---
Handler: None
Children: ['', 'about', 'sports', 'career']

Handler: career_handler
Children: []

--- Inserting:  ['/sports/climbing', 'sports_climbing_handler']  ---
Handler: None
Children: ['', 'about', 'sports', 'career']

Handler: sports_handler
Children: ['climbing']

Handler: sports_climbing_handler
Children: []

--- Inserting:  ['/sports/BJJ', 'sports_bjj_handler']  ---
Handler: None
Children: ['', 'about', 'sports', 'career']

Handler: sports_handler
Children: ['bjj', 'climbing']

Handler: sports_bjj_handler
Children: []


In [4]:
# Finding a path and respective handler
def find_path(path):
    print(routeTrie.find(path))

paths = ['/about', '/about/me', '/about/me/', '/about/me///', '', '/', '//', '/sports',
        '/career/', '/sports/bjj', '/sports/kayaking']

for path in paths:
    find_path(path)


--- Finding:  /about  ---
None

--- Finding:  /about/me  ---
about_me_handler

--- Finding:  /about/me/  ---
about_me_handler

--- Finding:  /about/me///  ---
None

--- Finding:    ---
home_handler

--- Finding:  /  ---
home_handler

--- Finding:  //  ---
home_handler

--- Finding:  /sports  ---
NEW_sports_handler

--- Finding:  /career/  ---
career_handler

--- Finding:  /sports/bjj  ---
sports_bjj_handler

--- Finding:  /sports/kayaking  ---
None


### Test Router

In [6]:
# Here are some test cases and expected outputs you can use to test your implementation

# create the router and add a route
router = Router("root handler", "not found handler") # remove the 'not found handler' if you did not implement this
router.add_handler("/home/about", "about handler")  # add a route

# some lookups with the expected output
print(router.lookup("/")) # should print 'root handler'
print(router.lookup("/home")) # should print 'not found handler' or None if you did not implement one
print(router.lookup("/home/about")) # should print 'about handler'
print(router.lookup("/home/about/")) # should print 'about handler' or None if you did not handle trailing slashes
print(router.lookup("/sports")) # should print 'not found handler' or None if you did not implement one
print(router.lookup("/home/about//")) # should print 'not found handler' or None if you did not implement one
print(router.lookup("/career/")) # should print 'not found handler' or None if you did not implement one

router.add_handler("/home/about/", "NEW About Handler")
print(router.lookup("/")) # should print 'root handler'
print(router.lookup("/home")) # should print 'not found handler' or None if you did not implement one
print(router.lookup("/home/about")) # should print 'about handler'
print(router.lookup("/home/about/")) # should print 'about handler' or None if you did not handle trailing slashes
print(router.lookup("/sports")) # should print 'not found handler' or None if you did not implement one
print(router.lookup("/home/about//")) # should print 'not found handler' or None if you did not implement one
print(router.lookup("/career/")) # should print 'not found handler' or None if you did not implement one


--- Inserting:  ['/', 'root handler']  ---
Handler: None
Children: ['']

Handler: root handler
Children: []

--- Inserting:  ['/home/about', 'about handler']  ---
Handler: None
Children: ['', 'home']

Handler: None
Children: ['about']

Handler: about handler
Children: []

--- Finding:  /  ---
root handler

--- Finding:  /home  ---
not found handler

--- Finding:  /home/about  ---
about handler

--- Finding:  /home/about/  ---
about handler

--- Finding:  /sports  ---
not found handler

--- Finding:  /home/about//  ---
not found handler

--- Finding:  /career/  ---
not found handler

--- Inserting:  ['/home/about/', 'NEW About Handler']  ---
Handler: None
Children: ['', 'home']

Handler: None
Children: ['about']

Handler: NEW About Handler
Children: []

--- Finding:  /  ---
root handler

--- Finding:  /home  ---
not found handler

--- Finding:  /home/about  ---
NEW About Handler

--- Finding:  /home/about/  ---
NEW About Handler

--- Finding:  /sports  ---
not found handler

--- Findin